In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq pyautogen==0.1.14 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [1]:
from getpass import getpass

from autogen import AssistantAgent, GroupChat, GroupChatManager, UserProxyAgent

In [2]:
OPENAI_API_KEY = getpass()

··········


In [3]:
gpt_config_list = [
    {
        # 'model': 'gpt-4',
        # 'model': "gpt-3.5-turbo",
        "model": "gpt-3.5-turbo-16k",
        "api_key": OPENAI_API_KEY,
    }
]

llm_config = {"config_list": gpt_config_list, "use_cache": False, "temperature": 0}

In [4]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="""A human admin. Interact with the planner to discuss the plan.
   Plan execution needs to be approved by this admin.""",
    code_execution_config=False,
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the
    code in a code block that specifies the script type. The user can't modify your code. So do not suggest
    incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by
    the executor. Don't include multiple code blocks in one response. Do not ask others to copy and paste the result.
    Check the execution result returned by the executor. If the result indicates there is an error, fix the error and
    output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed
    or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your
    assumption, collect additional info you need, and think of a different approach to try.""",
)

planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and Karen, until admin
    approval. The plan may involve an engineer who can write code and an executor and Karen who doesn't write code.
    Explain the plan first. Be clear which step is performed by an engineer, executor, and Karen.""",
    llm_config=llm_config,
)

executor = AssistantAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    code_execution_config={"last_n_messages": 3, "work_dir": "source"},
)

critic = AssistantAgent(
    name="Karen",
    system_message="Karen. Double check plan, claims, code from other agents and provide feedback (whine as a real-world Karen).",
    llm_config=llm_config,
)

groupchat = GroupChat(
    agents=[user_proxy, engineer, planner, executor, critic], messages=[], max_round=50
)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [5]:
message = """
Build a landing page for a SaaS that creates chatbots using ChatGPT from a set of documents.
Think of a name for the product and a short tagline. Add a form to collect user email
for when the product is ready. Add an admin page that shows a table of added emails.
Use flask for the app, Tailwind CSS for styling (must be centered and look good on mobile devices too),
and sqlite3 as a database.
"""

In [6]:
user_proxy.initiate_chat(manager, message=message)

Admin (to chat_manager):


Build a landing page for a SaaS that creates chatbots using ChatGPT from a set of documents.
Think of a name for the product and a short tagline. Add a form to collect user email
for when the product is ready. Add an admin page that shows a table of added emails.
Use flask for the app, Tailwind CSS for styling (must be centered and look good on mobile devices too), 
and sqlite3 as a database.


--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan:

Step 1: Naming and Tagline
- Executor: Come up with a name for the product and a short tagline that reflects its purpose and benefits.

Step 2: Design the Landing Page
- Engineer: Use Flask to create a landing page with a form to collect user emails.
- Engineer: Utilize Tailwind CSS to style the landing page, ensuring it is centered and looks good on mobile devices.

Step 3: Database Integration
- Engineer: Set up a SQLite3 database to store the collected